### Importing the required modules/packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import re
import nltk
from nltk.corpus import stopwords
import string
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
from mlxtend.feature_selection import ColumnSelector
from sklearn.compose import ColumnTransformer
nltk.download('stopwords')
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam




/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Loading file and looking into the dimensions of data

In [ ]:
raw_data = pd.read_csv("SMSSpamData.csv",names=['label','text'])
pd.set_option('display.max_colwidth',100)


print(f"Shape of Data --> {raw_data.shape}\n")
#print(pd.crosstab(raw_data['label'],columns = 'label'))
#pd.crosstab(raw_data['label'],columns = 'label',normalize=True)
raw_data.head()

Shape of Data --> (5572, 2)



,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [ ]:
### Label distrbution (Ham is messaged which are not Spam)
pd.crosstab(raw_data['label'],columns = 'label',normalize=True)
#raw_data.head()

col_0,label
label,
ham,0.865937
spam,0.134063


## **Data Processing**

### Functions to Create new features and cleaning the data

In [ ]:
## Creat punc
def punct_pc(text):
    punct_count = sum([1 for char in text if char in string.punctuation])
    return (punct_count/(len(text) - text.count(' ')))*100

def clean_data(text):
    punct = "".join([word.lower() for word in text if word not in string.punctuation])
    splt = re.split('\W+',punct)
    txt = [nltk.PorterStemmer().stem(word) for word in splt if word not in nltk.corpus.stopwords.words('english')]
    return txt


### Train and Test Split 

In [ ]:
## Splitting the Data using Test size 0.2
X_train,X_test,y_train,y_test = train_test_split(raw_data[['text',]],raw_data['label'],test_size=0.2,random_state=23)

In [ ]:
## Covert target to binary
y_train = pd.get_dummies(y_train, drop_first=True)
y_test = pd.get_dummies(y_test, drop_first=True)

In [ ]:
print(pd.crosstab(y_train,columns = 'label',normalize=True))
print(pd.crosstab(y_test,columns = 'label',normalize=True))
X_train.head()

col_0   label
label        
ham    0.8674
spam   0.1326
col_0    label
label         
ham    0.86009
spam   0.13991


,text
1627,Dear how you. Are you ok?
4608,Been running but only managed 5 minutes and then needed oxygen! Might have to resort to the roll...
2709,"Nah, I'm a perpetual DD"
639,"What will we do in the shower, baby?"
4544,I've been trying to reach him without success


In [ ]:
######################## Create new features Train and Test Data ########################
#- Two new features are created - 
#- 1) text_length (the total length of the text)
#- 2) Punct_pc (the percentage of punctuations in the text)

## Train Data
X_train["punct_pc"] = X_train["text"].apply(lambda x: punct_pc(x))
X_train["text_length"] = X_train["text"].apply(lambda x: len(x)-x.count(' '))

## Test Data
X_test["punct_pc"] = X_test["text"].apply(lambda x: punct_pc(x))
X_test["text_length"] = X_test["text"].apply(lambda x: len(x)-x.count(' '))



### Pipeline to process the Text Data

- Tokenization
- Cleaning
- Normalization
- Lemmatization
- Steaming

Custom Transformer to select the Columns

In [ ]:
## Customer transformer to Select features 
class ColumnExtractor(TransformerMixin):
    def __init__(self, cols):
        self.cols = cols

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xcols = pd.Series(X[self.cols])
        return (Xcols)


In [ ]:
import numpy as np
import multiprocessing as mp

import string
import spacy 
import en_core_web_sm
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator


nlp = en_core_web_sm.load()


class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 
                    variety="BrE",
                 user_abbrevs={},
                 n_jobs=1):
        """
        Text preprocessing transformer includes steps:
            1. Text normalization
            2. Punctuation removal
            3. Stop words removal
            4. Lemmatization
        
        variety - format of date (AmE - american type, BrE - british format) 
        user_abbrevs - dict of user abbreviations mappings (from normalise package)
        n_jobs - parallel jobs to run
        """
        self.variety = variety
        self.user_abbrevs = user_abbrevs
        self.n_jobs = n_jobs

    def fit(self, X, y=None):
        return self

    def transform(self, X, *_):
        X_copy = X.copy()

        partitions = 1
        cores = mp.cpu_count()
        if self.n_jobs <= -1:
            partitions = cores
        elif self.n_jobs <= 0:
            return X_copy.apply(self._preprocess_text)
        else:
            partitions = min(self.n_jobs, cores)

        data_split = np.array_split(X_copy, partitions)
        pool = mp.Pool(cores)
        data = pd.concat(pool.map(self._preprocess_part, data_split))
        pool.close()
        pool.join()

        return data

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        normalized_text = self._normalize(text)
        doc = nlp(normalized_text)

        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)
        return self._lemmatize(removed_stop_words)

    def _normalize(self, text):
        # some issues in normalise package
        try:
            return ' '.join(normalise(text, variety=self.variety, user_abbrevs=self.user_abbrevs, verbose=False))
        except:
            return text

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if not t.is_stop]

    def _lemmatize(self, doc):
        return ' '.join([t.lemma_ for t in doc])



num_cols = ["punct_pc","text_length"]
Column_trans = ColumnTransformer(
     [('scaler', StandardScaler(),num_cols)],
     remainder='drop')

num_pipe = Pipeline([
  ('scaler', Column_trans)
  ])

In [ ]:
Preprocess_text = Pipeline([("select_text", ColumnExtractor(cols="text")),
                            ('preprocess', TextPreprocessor())
                            ])

preprocessing_pipeline = Pipeline([
    ('feat_union', FeatureUnion(transformer_list=[
          ('text_pipeline', Preprocess_text),
          ('num_pipeline', num_pipe)
          ]))
    ])

**Preprocess the Train and Test data using the pipeline**

In [ ]:
X_train_preprocessed = Preprocess_text.fit_transform(X_train)
X_test_preprocessed = Preprocess_text.fit_transform(X_test)

Calculate the size of the vocabulary (i.e the number of unique words in the entire corpus) . This is needed as a paramter "input_dim" in the embedded layer 

In [ ]:
from collections import Counter
# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

counter = counter_word(X_train_preprocessed)
vocab_size = len(counter)

In [ ]:
# We are taking a maximum lenght for the sequence as 30 (sequence above this length will be trimmed down and 
# and below this lenght will be padded with zeroes)
max_length = 30

Using Tokenizer class from keras tokenize the sentences. Tokinzer assignes a unique ID for each word in the entire trainign set

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train_preprocessed)
X_train_sequences = tokenizer.texts_to_sequences(X_train_preprocessed)

In [ ]:
## We can check the index of all the words created by the tokenizer
word_index = tokenizer.word_index
print(word_index)

{'u': 1, '2': 2, 'ur': 3, 'go': 4, 'come': 5, 'gt': 6, '4': 7, 'lt': 8, 'not': 9, 'good': 10, 'know': 11, '£': 12, 'get': 13, 'free': 14, 'ok': 15, 'send': 16, 'day': 17, 'like': 18, 'want': 19, 'time': 20, 'love': 21, 'think': 22, 'text': 23, 'tell': 24, 'need': 25, 'r': 26, 'txt': 27, 'ü': 28, 'n': 29, 'reply': 30, 'sorry': 31, 'stop': 32, 'today': 33, 'da': 34, 'lor': 35, 'win': 36, '1': 37, 'home': 38, 'k': 39, 'mobile': 40, 'phone': 41, 's': 42, 'say': 43, 'hope': 44, 'later': 45, 'work': 46, 'ask': 47, 'hi': 48, 'new': 49, 'week': 50, 'night': 51, 'dear': 52, 'pls': 53, 'c': 54, 'hey': 55, 'message': 56, 'd': 57, 'thing': 58, 'wait': 59, 'meet': 60, 'miss': 61, 'claim': 62, 'great': 63, 'happy': 64, 'msg': 65, 'try': 66, 'oh': 67, 'leave': 68, 'wat': 69, 'feel': 70, 'number': 71, '3': 72, 'thank': 73, 'find': 74, 'way': 75, 'min': 76, 'friend': 77, 'yes': 78, 'late': 79, 'let': 80, 'tomorrow': 81, 'right': 82, 'www': 83, 'prize': 84, 'life': 85, 'e': 86, 'cash': 87, 'amp': 88, 'i

Using pad_squeence, pad the sequences to have the same length

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X_train_padded = pad_sequences(
    X_train_sequences, maxlen=max_length, padding="post", truncating="post"
)

In [ ]:
## Applying same tranformation to Test set
test_sequences = tokenizer.texts_to_sequences(X_test_preprocessed)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

## Deep Learning model training

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

model = Sequential()

model.add(Embedding(num_words, 32, input_length=max_length))
model.add(LSTM(64, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))


optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 32)            265984    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 290,881
Trainable params: 290,881
Non-trainable params: 0
_________________________________________________________________


In [ ]:
Trained_model = model.fit(
    X_train_padded, y_train, epochs=20, validation_data=(test_padded, y_test)
)

Epoch 1/20
140/140 [==============================] - 4s 25ms/step - loss: 1.8439e-05 - accuracy: 1.0000 - val_loss: 0.1550 - val_accuracy: 0.9839
Epoch 2/20
140/140 [==============================] - 4s 27ms/step - loss: 1.7119e-05 - accuracy: 1.0000 - val_loss: 0.1563 - val_accuracy: 0.9839
Epoch 3/20
140/140 [==============================] - 3s 24ms/step - loss: 1.5772e-05 - accuracy: 1.0000 - val_loss: 0.1576 - val_accuracy: 0.9839
Epoch 4/20
140/140 [==============================] - 3s 24ms/step - loss: 1.4583e-05 - accuracy: 1.0000 - val_loss: 0.1589 - val_accuracy: 0.9839
Epoch 5/20
140/140 [==============================] - 3s 24ms/step - loss: 1.3492e-05 - accuracy: 1.0000 - val_loss: 0.1601 - val_accuracy: 0.9839
Epoch 6/20
140/140 [==============================] - 3s 24ms/step - loss: 1.2482e-05 - accuracy: 1.0000 - val_loss: 0.1614 - val_accuracy: 0.9839
Epoch 7/20
140/140 [==============================] - 3s 23ms/step - loss: 1.1565e-05 - accuracy: 1.0000 - val_loss: 0